In [1]:
%load_ext autoreload
%autoreload 2
from copy import deepcopy
from textwrap import dedent
import os, re, json, ast
import pandas as pd
from pprint import pprint
from llm_api_calls import send_message_to_gemini_async, RateLimiter, send_message_open_router_async, send_message_to_openai_async
from algo import base_start_dialog_algorithm, simulate_user_response_prompt, \
    generate_readable_history_from_end,  update_message_history_with_system_message, convert_gemini_history_to_open_router, \
    base_dialog_disbacteriose_algorithm, extract_anamnes_data_prompt, student_card_template, get_abnormal_labs_prompt, extract_choosed_ferments_prompt
from koprogramma import koprogramma_df, how_to_take_ferments_df, suggest_ferments_prompt


# Start algo question answer scenario

In [3]:

class ConversationManager:
    def __init__(self, verbose={'usage': True, 'messages': True, 'errors': True}):
        self.rate_limiter = RateLimiter(13, 14)
        self.verbose = verbose

        # user info
        self.telegram_user_id = []

        # conversaion info
        self.message_history_dialog = []
        self.message_history_on_the_step = []
        self.usage_history = []
        self.dialog_count = 0
        self.dialog_status = None
        self.extraction_usage_history = []

        # extraction
        self.student_card = {}


    async def start_step_1(self):
        """
        Сбор анамнеза общие вопросы и желчный
        """
        self.dialog_status = 'STEP_1'
        self.student_card = student_card_template().copy()
        system_message = base_start_dialog_algorithm()#.replace("$STUDENT_CARD", str(self.student_card))
        self.message_history_on_the_step.append({"role": "user", "parts": [system_message], "meta_info": {'system_message':True}})
        first_model_answer = "Здравствуйте! Меня зовут Аика, и я очень рада приветствовать вас в нашем диалоге по улучшению здоровья. Прежде чем начать, мне необходимо проверить ваш промокод, чтобы получить доступ к созданию алгоритма. Ваш промокод находится в 9 уроке."
        self.message_history_on_the_step.append({"role": "model", "parts": [first_model_answer]})
        if self.verbose['messages']:
            print('\nSYSTEM MESSAGE:\n', system_message)
            print('\nMODEL first_model_answer:\n', first_model_answer)

    
    async def start_step_2(self):
        """
        Сбор анамнеза копрограмма и дисбиоз
        """
        self.dialog_status = 'STEP_2'
        self.message_history_on_the_step = []
        
        filled_card = str({key:value for key, value in self.student_card.items() if value != ''})
        PREVIOUS_CONVERSATION_SUMMARY = f"В предыдущей части диалога вы собрали анамнез по следующим полям {filled_card}"
        system_message = base_dialog_disbacteriose_algorithm()\
            .replace("$STUDENT_INFO", str("Иван, 35 лет"))\
            .replace("$PREVIOUS_CONVERSATION_SUMMARY", PREVIOUS_CONVERSATION_SUMMARY)
        self.message_history_on_the_step.append({"role": "user", "parts": [system_message], "meta_info": {'system_message':True}})
        first_model_answer = """Поздравляю, первый этап пройден! Теперь мы разберемся с твоими анализами на дисбактериоз и копрограмму.
        Для начала ознакомься с материалами по выбору пробиотиков https://tinyurl.com/mr33yju. 
        Если ты из зарубежных стран, тебе могут быть полезны эти материалы https://tinyurl.com/2a67rmzf и https://tinyurl.com/586vm8n7. 
        Скажи как будешь готов продолжить."""
        self.message_history_on_the_step.append({"role": "model", "parts": [first_model_answer]})
        if self.verbose['messages']:
            print('\nSYSTEM MESSAGE:\n', system_message)
            print('\nMODEL first_model_answer:\n', first_model_answer)


    async def start_step_3(self):
        """
        Выбор ферментов
        """
        self.dialog_status = 'STEP_3'
        self.message_history_on_the_step = []
        LABS_koprogramma, preparats_dic = await self.get_recommended_ferments_and_bad_koprogramma()

        PREVIOUS_CONVERSATION_SUMMARY = "В предыдущей части диалога вы собрали анамзен по копрограмме и дисбиозу а теперь приступаете к интерпретации копрограммы и подбору ферментов"
        system_message = suggest_ferments_prompt()\
            .replace("$LABS", str(LABS_koprogramma))\
            .replace("$DRUGS", str(preparats_dic))\
            .replace("$PREVIOUS_CONVERSATION_SUMMARY", PREVIOUS_CONVERSATION_SUMMARY)
        
        self.message_history_on_the_step.append({"role": "user", "parts": [system_message], "meta_info": {'system_message':True}})
        first_model_answer = """Пищеварительные ферменты - это вещества, которые помогают организму расщеплять пищу на более мелкие части, чтобы ее было легче усваивать. 
        Они вырабатываются в желудке, поджелудочной железе и тонком кишечнике. Скажи, ты уже изучил гайд и выбрал себе ферменты? Я могу тебе помочь с этим, если у тебя есть вопросы"""
        self.message_history_on_the_step.append({"role": "model", "parts": [first_model_answer]})
        if self.verbose['messages']:
            print('\nSYSTEM MESSAGE:\n', system_message)
            print('\nMODEL first_model_answer:\n', first_model_answer)


    async def get_recommended_ferments_and_bad_koprogramma(self):
        # GET ABNORMAL RESULTS OF LABS ANSALYSIS
        kop1 = koprogramma_df().copy()
        kop1['Результат анализов человека'] = kop1['Название анализа копрограммы'].map(self.student_card)
        skip_this_results = ['не обнаружено', 'не указано', None]
        kop1 = kop1[~(kop1['Результат анализов человека'].isin(skip_this_results))]
        if kop1.shape[0] > 0:
            LABS_koprogramma = str(kop1[["Название анализа копрограммы", "Вариант нормы",
                                        "Отклонение", "Результат анализов человека"]].to_dict(orient='records'))
        else:
            LABS_koprogramma = 'Копрограмма не показала отклонений'
        LABS_koprogramma = "Результаты копрограммы: " + LABS_koprogramma
        get_abnormal_labs_prompted = get_abnormal_labs_prompt().replace("$LABS", "Результаты копрограммы: " + str(LABS_koprogramma))
        history = [{'role':'user', 'parts': [get_abnormal_labs_prompted]}]
        response = await send_message_to_gemini_async(history, generation_params={"temperature": 0})
        try:
            kop_bad = json.loads(response.get('text_response'))
        except:
            print(f'****** ERROR ***** не получилось спарсить абнормальные результаты копрограммы', response.get('text_response'))
            kop_bad = []

        kop_bad_df = kop1[kop1['Название анализа копрограммы'].isin(kop_bad)].reset_index(drop=True)

        needed_ferments = {"Ферменты желудочного сока": kop_bad_df['Ферменты желудочного сока'].dropna().unique().tolist(),
                            "Ферменты поджелудочной железы": kop_bad_df['Ферменты поджелудочной железы'].dropna().unique().tolist(),
                            "Гепатопротекторы и желчегонные": kop_bad_df['Гепатопротекторы и желчегонные'].dropna().unique().tolist(),
                            "Антивоспалительный пребиотик": kop_bad_df['Антивоспалительный пребиотик'].dropna().unique().tolist(),
                            }

        preparats_dic = {}
        for p in needed_ferments:
            key_name = f'{p}: {needed_ferments[p]} для корректировки этих факторов:'
            drugs_list = kop_bad_df.groupby(p)['Название анализа копрограммы'].apply(list).values.tolist()
            if len(drugs_list) > 0:
                preparats_dic[key_name] = drugs_list[0]

        return LABS_koprogramma, preparats_dic


    async def process_model_interaction(self, user_query, provider='gemini'):
        # ADD USER MESSAGE TO HISTORY
        if user_query.strip() == '':
            user_query = "Собеседник отправил пустое сообщение. Расскажи уместную шутку чтобы разбавить диалог"
        self.message_history_on_the_step.append({"role": "user", "parts": [user_query]})
        
        if provider == 'gemini':
            response = await send_message_to_gemini_async(self.message_history_on_the_step, rate_limiter=self.rate_limiter, generation_params={"temperature": 0.1, "top_k":3})
        elif provider == 'router_sonnet':
            openrouter_history = convert_gemini_history_to_open_router(self.message_history_on_the_step)
            response = await send_message_open_router_async(openrouter_history, rate_limiter=self.rate_limiter, model="anthropic/claude-3-sonnet")
        elif provider == 'gpt-3.5-turbo':
            response = await send_message_to_openai_async(self.message_history_on_the_step, model='gpt-3.5-turbo', rate_limiter=self.rate_limiter, generation_params={"temperature": 0.1})
        elif provider == 'gpt-4-turbo-2024-04-09':
            response = await send_message_to_openai_async(self.message_history_on_the_step, model='gpt-4-turbo-2024-04-09', rate_limiter=self.rate_limiter, generation_params={"temperature": 0.1})
        else:
            raise "This provider do not exist error"
        
        
        input_tokens = response.get('input_tokens')
        output_tokens = response.get('output_tokens')
        answer_text = response.get('text_response')

        # ADD MODEL MESSAGE TO HISTORY
        self.message_history_on_the_step.append({"role": "model", "parts": [answer_text]})
        self.usage_history.append({"provider": provider, "query": user_query, "answer": answer_text, "input_tokens": input_tokens, "output_tokens": output_tokens})

        if self.verbose['messages']:
            print('\nUSER:\n', user_query)
            print('\nMODEL:\n', answer_text)
        if self.verbose['usage']:
            print({"provider": provider, "input_tokens": input_tokens, "output_tokens": output_tokens})

        return answer_text


    async def update_student_card_from_message_history(self, extraction_prompt=extract_anamnes_data_prompt(), every_n_turns=4, retry=1):
        answer_text = self.message_history_on_the_step[-1]['parts'][0]

        if (self.dialog_count % every_n_turns == 0 and self.dialog_count > 0) or ('GO_TO' in answer_text.upper()):
        
            readable_history = await generate_readable_history_from_end(messages=self.message_history_on_the_step, dialogs_num=every_n_turns+1)

            extract_student_health_data_prompted = extraction_prompt.replace("$DIALOG", readable_history)#.replace("$STUDENT_CARD", str(self.student_card))
            messages = [{"role": "user", "parts": [extract_student_health_data_prompted]}]
            response = await send_message_to_gemini_async(messages, rate_limiter=self.rate_limiter, generation_params={"temperature": 0, "top_k":1})

            input_tokens = response.get('input_tokens')
            output_tokens = response.get('output_tokens')
            answer_text = response.get('text_response')

            try:
                # UPDATE STUDENT CARD
                new_info = json.loads(answer_text.strip('```json').strip('```'))
                self.student_card.update(new_info)
            except:
                new_info = f'********* ВНИМАНИЕ ********* Ошибка извлечения информации: {response}'
                self.update_student_card_from_message_history(every_n_turns=4, retry=2)

            if self.verbose['messages']:
                print('NEW INFO:', new_info)

            # ADD TO HISTORY
            self.extraction_usage_history.append({"provider": 'gemini', "query": extract_student_health_data_prompted, "answer": answer_text, "input_tokens": input_tokens, "output_tokens": output_tokens})

    
    async def update_dialog_status(self):
        self.dialog_count += 1
        answer_text = self.message_history_on_the_step[-1]['parts'][0]
        
        if 'TERMINATE' in answer_text.upper():   
            self.dialog_status = 'TERMINATE'
        elif 'GO_TO_STEP_2' in answer_text.upper(): 
            self.dialog_status = 'GO_TO_STEP_2'
        elif 'GO_TO_STEP_3' in answer_text.upper(): 
            self.dialog_status = 'GO_TO_STEP_3'
        elif 'GO_TO_STEP_4' in answer_text.upper(): 
            self.dialog_status = 'GO_TO_STEP_4'


    async def simulate_user_response(self):

        user_query = simulate_user_response_prompt()\
            .replace("$DOCTOR_QUESTION", self.message_history_on_the_step[-1]['parts'][0])\
            .replace("$PREVIOUS_CONVERSATION", str(self.message_history_on_the_step[:-1]))
        
        history = [{'role': 'user', 'parts': [user_query]}]
        response_user = await send_message_to_gemini_async(history = history)
        user_query = response_user.get('text_response')
        return user_query
    
    
    async def send_message_to_user_telegram(self, answer_text):
        # self.user_id
        pass


    async def run_anamnes_step1(self, provider='gemini'):
        await self.start_step_1()
        while self.dialog_status not in ['TERMINATE', 'GO_TO_STEP_2']:
            if self.verbose: print(f'dialog_count: {self.dialog_count}')
            user_query = await self.simulate_user_response()
            # user_query = input()
            answer_text = await self.process_model_interaction(user_query, provider=provider) #  gpt-3.5-turbo gemini router_sonnet gpt-4-turbo-2024-04-09
            await self.send_message_to_user_telegram(answer_text)
            await self.update_dialog_status()
            await self.update_student_card_from_message_history()


    async def run_anamnes_step2(self, provider='gemini'):
        await self.start_step_2()
        while self.dialog_status not in ['TERMINATE', 'GO_TO_STEP_3']:
            if self.verbose: print(f'dialog_count: {self.dialog_count}')
            user_query = await self.simulate_user_response()
            # user_query = input()
            answer_text = await self.process_model_interaction(user_query, provider=provider) #  gpt-3.5-turbo gemini router_sonnet gpt-4-turbo-2024-04-09
            await self.send_message_to_user_telegram(answer_text)
            await self.update_dialog_status()
            await self.update_student_card_from_message_history()

    async def run_step3(self, provider='gemini'):
        await self.start_step_3()
        while self.dialog_status not in ['TERMINATE', 'GO_TO_STEP_4']:
            if self.verbose: print(f'dialog_count: {self.dialog_count}')
            user_query = await self.simulate_user_response()
            # user_query = input()
            answer_text = await self.process_model_interaction(user_query, provider=provider) #  gpt-3.5-turbo gemini router_sonnet gpt-4-turbo-2024-04-09
            await self.send_message_to_user_telegram(answer_text)
            await self.update_dialog_status()
            await self.update_student_card_from_message_history()

conv_manager = ConversationManager()

try:
    await conv_manager.run_anamnes_step1()
    await conv_manager.run_anamnes_step2()
    await conv_manager.run_step3()
except KeyboardInterrupt:
    print('Stopping due to KeyboardInterrupt')



SYSTEM MESSAGE:
 <SYSTEM_MESSAGE>
Ты Имя: Аинна (Аика для друзей)
Роль: Уважительная ИИ-ассистентка, специализирующаяся на сборе анамнеза (анализов). Вся коммуникация направлена на сбор анамнеза, чтобы по его результатам следующим этапам (это будешь делать уже не ты) ученик с другим ИИ-ассистентом собрал его алгоритм.
Основные качества: Понимающая, методичная, внимательная к деталям, терпеливая, образованная в области здоровья и благополучия.
Метод коммуникации: Сократовский метод, который предполагает задавать наводящие вопросы для поддержки самостоятельного мышления и закрепления знаний.
Правила общения:
1 - Если нужна помощь по сложным вопросам, то перенаправляй к врачу на курсе Марии Смирновой через Telegram @mariasmirnova03.
2 - Не откланяйся от плана беседы. Если ученик усиленно пытается отойти от плана беседы, то напиши <TERMINATE>

Вот план беседы. Спрашивай эти пункты у ученика по очереди. Задавай вопросы по одному вопросу за раз. Если диалог отошел от плана, то напомни о тво

In [3]:
## ШАГ 1

conv_manager = ConversationManager()
await conv_manager.start_step_1()
while conv_manager.dialog_status not in ['TERMINATE', 'GO_TO_STEP_2']:
    if conv_manager.verbose: print(f'dialog_count: {conv_manager.dialog_count}')
    user_query = await conv_manager.simulate_user_response()
    # user_query = input()
    answer_text = await conv_manager.process_model_interaction(user_query, provider='gemini') #  gpt-3.5-turbo gemini router_sonnet gpt-4-turbo-2024-04-09
    await conv_manager.send_message_to_user_telegram(answer_text)
    await conv_manager.update_dialog_status()
    await conv_manager.update_student_card_from_message_history()

student_card = conv_manager.student_card
student_card

dialog_count: 0

USER:
 masgutovai007

MODEL:
 Благодарю вас за предоставление промокода. Теперь я могу приступить к сбору информации, необходимой для создания вашего индивидуального алгоритма.

1. Могу я узнать ваше имя и возраст?
{'provider': 'gemini', 'input_tokens': 7, 'output_tokens': 42}
dialog_count: 1

USER:
 Ирина, 35 лет

MODEL:
 Благодарю вас за предоставление этой информации.

2. Каких целей по здоровью вы хотите достичь на этом курсе?
{'provider': 'gemini', 'input_tokens': 6, 'output_tokens': 27}
dialog_count: 2

USER:
 Я хочу улучшить свое общее самочувствие и снизить риск хронических заболеваний.

MODEL:
 Похвально, что вы ставите перед собой такие важные цели.

3. Какое у вас артериальное давление? Относитесь ли вы к гипотоникам (менее 110/70), нормотоникам (110/70 - 130/90) или гипертоникам (более 130/90)?
{'provider': 'gemini', 'input_tokens': 18, 'output_tokens': 82}
dialog_count: 3

USER:
 Нормотоник, обычно 120/80.

MODEL:
 Благодарю вас за предоставление этой инфо

{'Промокод': 'masgutovai007',
 'Имя и возраст': 'Ирина, 35 лет',
 'Цели по здоровью': 'Улучшить общее самочувствие и снизить риск хронических заболеваний',
 'Артериальное давление': 'нормотоник 120/80',
 'Тип сокращения желчного пузыря': 'желчный пузырь удален',
 'Выбранная желчегонная трава': 'Пижма',
 'Желчегонная гимнастика': 'В течение недели начну выполнять желчегонную гимнастику',
 'Витамин Д': 'не ответил',
 'Время принятия лимфатического душа': 'утром',
 'Лактобактерии (Молочнокислые бактерии)': '',
 'Бифидобактрии': '',
 'Кишечные палочки с типичными свойствами': '',
 'Энтерококки': '',
 'Энтеробактерии': '',
 'Неферментирующие бактерии': '',
 'Стафилококки': '',
 'Кишечные палочки с гемолитическими признаками': '',
 'Кишечные палочки \xa0со сниженными ферментативными признаками (лактоз-отрицательные кишечные палочки)': '',
 'Анаэробные бактерии (клостридии)': '',
 'Грибы (Кандиды)': '',
 'Бактериофаги к которым чувствительны микроорганизмы': '',
 'Щелочной рН кала': '',
 'Изб

In [4]:
## ШАГ 2 - Сбор анамнеза на дизбиоз и копрограмму

await conv_manager.start_step_2()
while conv_manager.dialog_status not in ['terminate', 'GO_TO_STEP_3']:
    if conv_manager.verbose: print(f'dialog_count: {conv_manager.dialog_count}')
    user_query = await conv_manager.simulate_user_response()
    # user_query = input()
    answer_text = await conv_manager.process_model_interaction(user_query, provider='gemini') #  gpt-3.5-turbo gemini router_sonnet gpt-4-turbo-2024-04-09
    await conv_manager.send_message_to_user_telegram(answer_text)
    await conv_manager.update_dialog_status()
    await conv_manager.update_student_card_from_message_history()

student_card = conv_manager.student_card
student_card

dialog_count: 9

USER:
 Я ознакомлюсь с материалами и дам вам знать, когда буду готов продолжить.

MODEL:
 Хорошо, я буду ждать твоего сообщения.
{'provider': 'gemini', 'input_tokens': 19, 'output_tokens': 9}
dialog_count: 10

USER:
 Хорошо, я дам вам знать, как только они будут готовы.

MODEL:
 Хорошо, я буду ждать твоего сообщения.
{'provider': 'gemini', 'input_tokens': 13, 'output_tokens': 9}
dialog_count: 11

USER:
 Хорошо, я напишу вам, как только будут результаты.

MODEL:
 Хорошо, я буду ждать твоего сообщения.
{'provider': 'gemini', 'input_tokens': 12, 'output_tokens': 9}
NEW INFO: {}
dialog_count: 12

USER:
 Перенесём на завтра

MODEL:
 Хорошо, я буду ждать твоего сообщения завтра.
{'provider': 'gemini', 'input_tokens': 6, 'output_tokens': 10}
dialog_count: 13

USER:
 Благодарю вас, доктор. Я обязательно напишу вам.

MODEL:
 Пожалуйста, не называй меня доктором, я всего лишь ИИ-ассистент. Я буду ждать твоего сообщения завтра.
{'provider': 'gemini', 'input_tokens': 12, 'output_t

In [2]:
## ШАГ 3 - Подбор ферментов и пробиотиков

conv_manager = ConversationManager()
conv_manager.student_card = {'Промокод': 'masgutovai007',
 'Имя и возраст': 'Святослав, 42 года',
 'Цели по здоровью': 'Восстановить свое прежнее состояние до возникновения проблем с печенью',
 'Артериальное давление': 'нормотоник 125/85',
 'Тип сокращения желчного пузыря': 'Гипокинетический тип',
 'Выбранная желчегонная трава': 'Кукурузные рыльца',
 'Желчегонная гимнастика': 'не указано',
 'Витамин Д': 'Принимаю 2000 МЕ',
 'Время принятия лимфатического душа': 'Утром',
 'Лактобактерии (Молочнокислые бактерии)': 'Слегка снижены',
 'Бифидобактрии': 'В норме',
 'Кишечные палочки с типичными свойствами': 'В норме',
 'Энтерококки': 'В норме',
 'Энтеробактерии': 'не указано',
 'Неферментирующие бактерии': 'не указано',
 'Стафилококки': 'не указано',
 'Кишечные палочки с гемолитическими признаками': 'не указано',
 'Кишечные палочки \xa0со сниженными ферментативными признаками (лактоз-отрицательные кишечные палочки)': 'не указано',
 'Анаэробные бактерии (клостридии)': 'не указано',
 'Грибы (Кандиды)': 'не указано',
 'Бактериофаги к которым чувствительны микроорганизмы': '',
 'Щелочной рН кала': 'в норме',
 'Избыток слизи': 'отсутствует',
 'Остатки непереваренной пищи': 'немного свеклы',
 'Мышечные волокна неизмененные (с исчерченностью)': 'не обнаружено',
 'Растительная клетчатка перевариваемая': 'в норме',
 'Жир нейтральный': 'незначительное количество',
 'Жирные кислоты': 'не обнаружено',
 'Мыла': 'не обнаружено',
 'Краxмал': 'не обнаружено',
 'Краxмал  внутриклеточный': 'не обнаружено',
 'Крахмал внеклеточный': 'не обнаружено',
 'Кристаллы триппельфосфаты': 'немного',
 'Кристаллы Шарко–Лейдена': 'не обнаружено',
 'Йодофильная флора': 'отсутствует',
 'Клостридии': 'не обнаружено',
 'Эпителий': 'в норме',
 'Выбранные ферменты': 'не указано',
 'Выбранные пробиотки': 'не указано'}

await conv_manager.start_step_3()
while conv_manager.dialog_status not in ['terminate', 'GO_TO_STEP_4']:
    if conv_manager.verbose: print(f'dialog_count: {conv_manager.dialog_count}')
    # user_query = await conv_manager.simulate_user_response()
    user_query = input()
    answer_text = await conv_manager.process_model_interaction(user_query, provider='gemini') #  gpt-3.5-turbo gemini router_sonnet gpt-4-turbo-2024-04-09
    await conv_manager.send_message_to_user_telegram(answer_text)
    await conv_manager.update_dialog_status()
    await conv_manager.update_student_card_from_message_history(extraction_prompt=extract_choosed_ferments_prompt())

student_card = conv_manager.student_card
student_card

NameError: name 'ConversationManager' is not defined

In [1]:
{'Выбранные ферменты': 'Энтеросан или Эвензим'}

{'Выбранные ферменты': 'Энтеросан или Эвензим'}

In [134]:
student_card = conv_manager.student_card
message_history = conv_manager.message_history
usage_history = conv_manager.usage_history
extraction_usage_history = conv_manager.extraction_usage_history
# folder = 'test_cases/2/'
# if not os.path.exists(folder): os.makedirs(folder)
# with open(f'{folder}/student_card.json', 'w') as file:
#     json.dump(student_card, file, indent=4, ensure_ascii=False)
student_card

{'Промокод': 'masgutovai007',
 'Имя и возраст': 'Святослав, 42 года',
 'Цели по здоровью': 'Восстановить свое прежнее состояние до возникновения проблем с печенью',
 'Артериальное давление': 'нормотоник 125/85',
 'Тип сокращения желчного пузыря': 'Гипокинетический тип',
 'Выбранная желчегонная трава': 'Кукурузные рыльца',
 'Желчегонная гимнастика': 'не указано',
 'Витамин Д': 'Принимаю 2000 МЕ',
 'Время принятия лимфатического душа': 'Утром',
 'Лактобактерии (Молочнокислые бактерии)': 'Слегка снижены',
 'Бифидобактрии': 'В норме',
 'Кишечные палочки с типичными свойствами': 'В норме',
 'Энтерококки': 'В норме',
 'Энтеробактерии': 'не указано',
 'Неферментирующие бактерии': 'не указано',
 'Стафилококки': 'не указано',
 'Кишечные палочки с гемолитическими признаками': 'не указано',
 'Кишечные палочки \xa0со сниженными ферментативными признаками (лактоз-отрицательные кишечные палочки)': 'не указано',
 'Анаэробные бактерии (клостридии)': 'не указано',
 'Грибы (Кандиды)': 'не указано',
 'Ба

In [17]:
import pandas as pd
u = pd.DataFrame(usage_history)
print(u['input_tokens'].sum(), u['output_tokens'].sum(), )
u

247 540


,provider,query,answer,input_tokens,output_tokens
0,gemini,Я ознакомился с материалами. Готов продолжить.,Хорошо. Давай начнем с анализа на дисбактериоз...,13,65
1,gemini,Нет значительных отклонений.,Хорошо. А есть ли в анализе превышение количес...,6,104
2,gemini,Уровень энтеробактерий и стафилококков повышен...,Понял. А обнаружены ли в анализе какие-либо па...,23,34
3,gemini,"К счастью, нет, патогенных микроорганизмов не ...",Хорошо. Есть ли у тебя результат на определени...,17,21
4,gemini,"Да, есть.",Отлично. Можешь прислать мне этот результат? Я...,4,23
5,gemini,Уже отправил.,Спасибо. Давай посмотрим. К каким бактериофага...,4,72
6,gemini,S: Enterobacteria phage ΦX174\nR: Pseudomonada...,Хорошо. Теперь давай перейдем к результатам ко...,20,55
7,gemini,Ознакомился с гайдом. Приступайте.,Отлично. Что из перечисленного ниже нашли в тв...,12,161
8,gemini,- Щелочной рН кала - присутствует в умеренном ...,<NEXT_STEP>,148,5


In [25]:

how_to_take_ferments_df()


,Препарат,Применение и дози
0,Ацидинпепсин,"По 500 мг 3-4 раза в сутки, во время или после..."
1,Энтеросан,За 15-20 мин до еды с небольшим количеством во...
2,Панкреатин,"Дозы препарата подбирают индивидуально, в зави..."
3,Консумед,"Дозы препарата подбирают индивидуально, в зави..."
4,Эвензим,"Дозы препарата подбирают индивидуально, в зави..."
5,Гепатосан,"За 15-20 минут до еды, запивая небольшим колич..."
6,Урсодезоксихолевая кислота,По 500 мг перед сном 1 раз в сутки в течение 2...
7,Закофальк,"Взрослым — по 3-4 табл. в день, до еды, не раз..."
8,Зостерин Ультра 60,1г/день 1г - вечером через 2ч после ужина\nКур...


In [27]:
koprogramma_df()

,Результат копрограммы,Ферменты желудочного сока,Ферменты поджелудочной железы,Гепатопротекторы и холекинетики,Анти- и пробиотики,Сорбенты
0,Щелочной рН кала,Ацидинпепсин или Энтеросан,Панкреатин или Консумед или Эвензим,None,None,Зостерин Ультра 60
1,Избыток слизи,None,None,None,Закофальк,Зостерин Ультра 60
2,Остатки непереваренной пищи,Ацидинпепсин или Энтеросан,Панкреатин или Консумед или Эвензим,None,None,Зостерин Ультра 60
3,Мышечные волокна неизмененные (с исчерченностью),Ацидинпепсин или Энтеросан,Панкреатин или Консумед или Эвензим,None,None,Зостерин Ультра 60
4,Растительная клетчатка перевариваемая,Ацидинпепсин или Энтеросан,None,Урсодезоксихолевая кислота или Гепатосан,Закофальк,Зостерин Ультра 60
5,Жир нейтральный,None,Панкреатин или Консумед или Эвензим,Урсодезоксихолевая кислота или Гепатосан,Закофальк,Зостерин Ультра 60
6,Жирные кислоты,None,None,Урсодезоксихолевая кислота или Гепатосан,Закофальк,Зостерин Ультра 60
7,Мыла,None,Панкреатин или Консумед или Эвензим,None,None,Зостерин Ультра 60
8,Краxмал,Ацидинпепсин или Энтеросан,None,None,None,Зостерин Ультра 60
9,Краxмал внутриклеточный,None,Панкреатин или Консумед или Эвензим,None,Закофальк,Зостерин Ультра 60


In [23]:
print(koprogramma_df()['Результат копрограммы'].values)

['Щелочной рН кала' 'Избыток слизи' 'Остатки непереваренной пищи'
 'Мышечные волокна неизмененные (с исчерченностью)'
 'Растительная клетчатка перевариваемая' 'Жир нейтральный'
 'Жирные кислоты' 'Мыла' 'Краxмал' 'Краxмал  внутриклеточный'
 'Крахмал внеклеточный' 'Кристаллы триппельфосфаты'
 'Кристаллы Шарко–Лейдена' 'Йодофильная флора' 'Клостридии' 'Эпителий']


# USAGES

In [10]:
# SONNET
import pandas as pd
us = pd.DataFrame(usage_history)
print("us['input_tokens'].sum()", us['input_tokens'].sum())
print("us['output_tokens'].sum()", us['output_tokens'].sum())
print("price", us['input_tokens'].sum()/1000 * 0.003 + us['output_tokens'].sum()/1000 * 0.015)
us

us['input_tokens'].sum() 15205
us['output_tokens'].sum() 1016
price 0.060855000000000006


,provider,query,answer,input_tokens,output_tokens
0,router_sonnet,"Здравствуйте, Аика. К сожалению, у меня нет пр...","Хорошо, в таком случае давайте продолжим наш д...",1206,52
1,router_sonnet,"Максим, 32 года.","Отлично, Максим. Расскажите, каких целей по ул...",1270,46
2,router_sonnet,Повысить общий уровень здоровья и продолжить в...,"Понятно, отличная цель! Давайте начнем с оценк...",1347,103
3,router_sonnet,Нормотоники,"Хорошо, спасибо за информацию. Следующий важны...",1459,118
4,router_sonnet,"Да, делал УЗИ органов брюшной полости и желчно...","Отлично, это хорошие результаты. Теперь скажит...",1640,98
5,router_sonnet,"Вы правы, доктор. Я выбрала тысячелистник для ...","Замечательно, Максим! Ваш выбор тысячелистника...",1774,172
6,router_sonnet,"Камней нет, я буду выполнять гимнастику вечеро...",Отличный выбор времени для желчегонной гимнаст...,1992,149
7,router_sonnet,"Да, сдавал. Результат показал 29 нг/мл. Приним...","Понятно, спасибо за информацию о витамине D. 2...",2185,135
8,router_sonnet,"Утром, перед работой.","Отлично, принятие лимфатического душа утром пе...",2332,143


In [14]:
# OPENAI
import pandas as pd
us = pd.DataFrame(usage_history)
print("us['input_tokens'].sum()", us['input_tokens'].sum())
print("us['output_tokens'].sum()", us['output_tokens'].sum())
print("price", us['input_tokens'].sum()/1000 * 0.01 + us['output_tokens'].sum()/1000 * 0.03)
us

us['input_tokens'].sum() 20196
us['output_tokens'].sum() 1523
price 0.24765000000000004


,provider,query,answer,input_tokens,output_tokens
0,gpt-4-turbo-2024-04-09,L847-LP9B-8KY63.,"Спасибо за предоставление промокода! Теперь, д...",1241,84
1,gpt-4-turbo-2024-04-09,"Меня зовут Джессика, мне 32 года.","Приятно познакомиться, Джессика! Теперь давайт...",1351,89
2,gpt-4-turbo-2024-04-09,{Я хотела бы улучшить свою физическую форму и ...,"Отличные цели, Джессика! Улучшение физической ...",1528,191
3,gpt-4-turbo-2024-04-09,"У меня нормальное артериальное давление, 120/80.","Спасибо за информацию, Джессика. Ваше давление...",1749,184
4,gpt-4-turbo-2024-04-09,Нормокинетический.,"Отлично, что у вас нормокинетический тип сокра...",1950,137
5,gpt-4-turbo-2024-04-09,"Да, тысячелистник.","Прекрасный выбор, Джессика! Тысячелистник отли...",2106,144
6,gpt-4-turbo-2024-04-09,"Да, УЗИ брюшной полости показало отсутствие ка...","Это отличные новости, Джессика! Отсутствие кам...",2296,142
7,gpt-4-turbo-2024-04-09,Вечером.,"Отличный выбор времени, Джессика. Вечером упра...",2451,156
8,gpt-4-turbo-2024-04-09,"Да, сдавала. Планирую принимать 2000 МЕ в день.","Хорошо, что вы контролируете уровень витамина ...",2641,229
9,gpt-4-turbo-2024-04-09,Вечером.,"Прекрасно, вечернее время идеально подходит дл...",2883,167


In [54]:
# GEMINI
us.to_csv('gemini_token_usage_16msgs.csv')
us

,input_tokens,output_tokens,conversation_tokens
0,979,47,1026
1,18,30,1074
2,4,51,1129
3,11,39,1179
4,14,87,1280
5,3,60,1343
6,5,86,1434
7,6,54,1494
8,4,141,1639
9,17,108,1764
